In [ ]:
# Create Stash app client

import pandas as pd
import dotenv

from libraries.client_stashapp import get_stashapp_client

dotenv.load_dotenv()

stash = get_stashapp_client()

In [ ]:
import os
import requests

tpdb_headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ['TPDB_API_KEY']}",
}

def get_scene_tpdb_id(stash_scene):
    # Extract TPDB ID from stash_ids
    tpdb_id = None
    for stash_id in stash_scene.get('stash_ids', []):
        if stash_id['endpoint'] == 'https://theporndb.net/graphql':
            tpdb_id = stash_id['stash_id']
            break
    return tpdb_id

def get_tpdb_scene_data(tpdb_id):
    url = f"https://api.theporndb.net/scenes/{tpdb_id}"
    response = requests.get(url, headers=tpdb_headers)
    return response.json()

# Function to convert seconds to MM:SS or HH:MM:SS format
def format_time(seconds):
    minutes, secs = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours:02d}:{minutes:02d}:{secs:02d}"
    else:
        return f"{minutes:02d}:{secs:02d}"


In [ ]:
marker_source_tpdb_tag = stash.find_tag("Marker Source: TPDB")
marker_source_skier_tag = stash.find_tag("Marker Source: Skier AI")
marker_source_manual_tag = stash.find_tag("Marker Source: Manual")
markers_tpdb_missing_tag = stash.find_tag("Markers: TPDB: Missing")
markers_tpdb_saved_tag = stash.find_tag("Markers: TPDB: Saved")
markers_timestamp_trade_missing_tag = stash.find_tag("Markers: timestamp.trade: Missing")
markers_timestamp_trade_saved_tag = stash.find_tag("Markers: timestamp.trade: Saved")
marker_source_timestamp_trade_tag = stash.find_tag("Marker Source: timestamp.trade")

# Check if all tags are truthy and print warnings if not
tags_to_check = [
    marker_source_tpdb_tag,
    marker_source_skier_tag,
    marker_source_manual_tag,
    markers_tpdb_missing_tag,
    markers_tpdb_saved_tag,
    markers_timestamp_trade_missing_tag,
    markers_timestamp_trade_saved_tag,
    marker_source_timestamp_trade_tag
]

for tag in tags_to_check:
    if not tag:
        print(f"Warning: Tag '{tag['name']}' not found in Stash.")


# Batch operation

In [ ]:
scenes_with_tpdb_id = stash.find_scenes({
    "stash_id_endpoint": {"endpoint": "https://theporndb.net/graphql", "modifier": "NOT_NULL" },
    "tags": {
        "value": [],
        "modifier": "INCLUDES_ALL",
        "excludes": [markers_tpdb_missing_tag['id'], markers_tpdb_saved_tag['id']]
    }
}, { "per_page": 250, "page": 1 })


In [ ]:
# Create markers from TPDB
scenes_updated_with_markers = 0
scenes_with_missing_markers = 0
total_scenes_processed = 0
tag_cache = {}  # Cache for tag lookups

def get_or_create_tag(title):
    if title in tag_cache:
        return tag_cache[title]
    
    tag = stash.find_tag(title)
    if not tag:
        # Retry with _TPDB suffix
        tpdb_title = f"{title}_TPDB"
        tag = stash.find_tag(tpdb_title)
        if not tag:
            # Create new tag with _TPDB suffix
            tag = stash.create_tag({
                "name": tpdb_title,
                "description": f"Marker from TPDB. Original name: {title}",
            })
            print(f"Created tag '{tpdb_title}' in Stash")
    
    tag_cache[title] = tag
    return tag

for scene in scenes_with_tpdb_id:
    total_scenes_processed += 1
    tpdb_id = get_scene_tpdb_id(scene)
    tpdb_scene_data = get_tpdb_scene_data(tpdb_id)
    if 'data' not in tpdb_scene_data:
        print(f"Error: 'data' key not found in TPDB scene data for scene ID: {scene['id']}")
        continue
    tpdb_markers = tpdb_scene_data['data'].get('markers', [])
    scene_markers = scene['scene_markers']

    # Convert Stash markers to a set of (title, seconds) tuples for easy comparison
    stash_markers_set = set((marker['title'], marker['seconds']) for marker in scene_markers)

    # If TPDB does not have any markers, add markers_tpdb_missing_tag to the scene
    if not tpdb_markers:
        # Re-read the scene from Stash to get the most up-to-date data
        updated_scene = stash.find_scene(scene['id'])
        # Ensure we're using the most recent tag IDs
        current_tag_ids = [tag['id'] for tag in updated_scene['tags']]
        # Add the missing tag ID if it's not already present
        if markers_tpdb_missing_tag['id'] not in current_tag_ids:
            current_tag_ids.append(markers_tpdb_missing_tag['id'])
        stash.update_scene({
            "id": scene['id'],
            "tag_ids": current_tag_ids
        })
        print(f"Added 'Markers: TPDB: Missing' tag to scene ID: {scene['id']}, Title: {scene['title']}")
        scenes_with_missing_markers += 1
        continue  # Skip to the next scene

    # Find markers in TPDB that are not in Stash
    new_markers = []
    for tpdb_marker in tpdb_markers:
        title = tpdb_marker['title']
        seconds = tpdb_marker['start_time']
        if (title, seconds) not in stash_markers_set:
            tag = get_or_create_tag(title)
            tpdb_marker['tag_id'] = tag['id']
            new_markers.append(tpdb_marker)

    if new_markers:
        scenes_updated_with_markers += 1
        print(f"Scene ID: {scene['id']}, Title: {scene['title']}, Total new markers to add: {len(new_markers)}")

        for new_marker in new_markers:
            stash.create_scene_marker({
                "scene_id": scene['id'],
                "title": new_marker['title'],
                "primary_tag_id": new_marker['tag_id'],
                "seconds": new_marker['start_time'],
                "tag_ids": [marker_source_tpdb_tag['id']],
            })
        
    # Add 'Markers: TPDB: Saved' tag if there are TPDB markers and the scene is missing this tag
    if tpdb_markers:
        updated_scene = stash.find_scene(scene['id'])
        current_tag_ids = [tag['id'] for tag in updated_scene['tags']]
        if markers_tpdb_saved_tag['id'] not in current_tag_ids:
            current_tag_ids.append(markers_tpdb_saved_tag['id'])
            stash.update_scene({
                "id": scene['id'],
                "tag_ids": current_tag_ids
            })
            print(f"Added 'Markers: TPDB: Saved' tag to scene ID: {scene['id']}, Title: {scene['title']}")

print(f"Total scenes processed: {total_scenes_processed}")
print(f"Scenes updated with markers: {scenes_updated_with_markers}")
print(f"Scenes with missing TPDB markers: {scenes_with_missing_markers}")


# timestamp.trade

In [ ]:
# Read all scenes from Stash with unknown timestamp.trade status
import requests

def get_stashdb_id(scene):
    for stash_id in scene.get('stash_ids', []):
        if stash_id['endpoint'] == 'https://stashdb.org/graphql':
            return stash_id['stash_id']
    return None

def fetch_markers_from_timestamp_trade(stashdb_id):
    url = f"https://timestamp.trade/get-markers/{stashdb_id}"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        return result if result != {} else None
    return None

# Read all scenes from Stash with stash_id_endpoint criteria
all_scenes = []
page = 1
max_pages = 10
per_page = 100  # Adjust this value based on your Stash instance's capabilities

while True:
    scenes_batch = stash.find_scenes(
        {
            "stash_id_endpoint": {"endpoint": "https://stashdb.org/graphql", "modifier": "NOT_NULL"},
            "tags": {
                "value": [],
                "modifier": "INCLUDES_ALL",
                "excludes": [markers_timestamp_trade_missing_tag['id'], markers_timestamp_trade_saved_tag['id']]
            }
        },
        filter={"per_page": per_page, "page": page}
    )
    if not scenes_batch:
        break
    all_scenes.extend(scenes_batch)
    page += 1
    print(f"Read {len(all_scenes)} scenes from Stash")
    if page > max_pages:
        break

df_scenes_with_stashdb_id = pd.DataFrame(all_scenes)

In [ ]:
# Read metadata from timestamp.trade for all matching scenes
from concurrent.futures import ThreadPoolExecutor

def process_scene(index, scene):
    stashdb_id = get_stashdb_id(scene)
    markers = fetch_markers_from_timestamp_trade(stashdb_id)
    return index, markers

df_scenes_with_stashdb_id['timestamp_trade_markers'] = None

results = []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_scene, index, scene) for index, scene in df_scenes_with_stashdb_id.iterrows()]
    for i, future in enumerate(futures):
        index, markers = future.result()
        results.append((index, markers))
        
        # Print progress every 100 scenes
        if (i + 1) % 100 == 0:
            print(f"Read markers from timestamp.trade for {i + 1} scenes")

# Update the DataFrame after all threads have completed
for index, markers in results:
    df_scenes_with_stashdb_id.at[index, 'timestamp_trade_markers'] = markers

# Create markers columns from timestamp.trade data
df_scenes_with_stashdb_id['markers'] = df_scenes_with_stashdb_id['timestamp_trade_markers'].apply(lambda x: x.get('marker', []) if isinstance(x, dict) else [])
df_scenes_with_stashdb_id['has_markers'] = df_scenes_with_stashdb_id['markers'].apply(lambda x: len(x) > 0)

In [ ]:
# Add Markers: timestamp.trade: Missing tag to scenes with no markers
df_scenes_without_markers = df_scenes_with_stashdb_id[~df_scenes_with_stashdb_id['has_markers']]

for index, scene in df_scenes_without_markers.iterrows():
    scene_id = scene['id']
    updated_scene = stash.find_scene(scene_id)
    current_tag_ids = [tag['id'] for tag in updated_scene['tags']]
    if markers_timestamp_trade_missing_tag['id'] not in current_tag_ids:
        current_tag_ids.append(markers_timestamp_trade_missing_tag['id'])
        stash.update_scene({
            "id": scene_id,
            "tag_ids": current_tag_ids
        })
        print(f"Added 'Markers: timestamp.trade: Missing' tag to scene ID: {scene_id}, Title: {scene['title']}")

In [ ]:
# Match timestamp.trade markers to Stash tags

# Create a DataFrame with distinct name and tag pairs
distinct_pairs = set()

for markers in df_scenes_with_stashdb_id['markers']:
    for marker in markers:
        name = marker.get('name', '')
        tag = marker.get('tag', '')
        distinct_pairs.add((name, tag))

# Create a DataFrame with distinct name and tag pairs
df_distinct_values = pd.DataFrame(list(distinct_pairs), columns=['name', 'tag'])

# Sort the DataFrame for better readability
df_distinct_values = df_distinct_values.sort_values(['name', 'tag']).reset_index(drop=True)


df_distinct_values['matching_name_tags'] = None
df_distinct_values['matching_tag_tags'] = None

for index, row in df_distinct_values.iterrows():
    matching_name_tags = stash.find_tag(row['name']) if row['name'] else None
    matching_tag_tags = stash.find_tag(row['tag']) if row['tag'] else None
    df_distinct_values.at[index, 'matching_name_tags'] = matching_name_tags
    df_distinct_values.at[index, 'matching_tag_tags'] = matching_tag_tags
    
    # Print progress every 50 rows
    if (index + 1) % 50 == 0:
        print(f"Processed {index + 1} rows")


In [ ]:
# Save timestamp.trade markers to Stash
processed_scenes = 0
for index, sample_row in df_scenes_with_stashdb_id[df_scenes_with_stashdb_id['has_markers']].iterrows():
    # Extract scene ID and markers
    scene_id = sample_row['id']
    markers = sample_row['markers']
    
    # Read scene data using stash.find_scene to verify that duplicates won't be created
    existing_scene = stash.find_scene(scene_id)
    
    # Create a list to store the marker data structures
    marker_data = []
    
    for marker in markers:
        name = marker.get('name', '')
        tag = marker.get('tag', '')
        start_time = marker.get('start', 0) / 1000.0
        
        # Find the corresponding row in df_distinct_values
        matching_row = df_distinct_values[(df_distinct_values['name'] == name) & (df_distinct_values['tag'] == tag)]
        
        if not matching_row.empty:
            # Get the matching tag ID from either name or tag column
            matching_tag = matching_row['matching_name_tags'].iloc[0] or matching_row['matching_tag_tags'].iloc[0]
            
            if matching_tag:
                # Create the marker data structure
                marker_data.append({
                    'scene_id': scene_id,
                    'tag_id': matching_tag['id'],
                    'tag_name': matching_tag['name'],
                    'start_time': start_time
                })
    
    for data in marker_data:
        # Check if the marker already exists in the scene
        marker_exists = False
        for marker in existing_scene['scene_markers']:
            if marker['primary_tag']['id'] == data['tag_id'] and abs(marker['seconds'] - data['start_time']) <= 5:
                marker_exists = True
                if marker_source_timestamp_trade_tag['id'] not in [tag['id'] for tag in marker['tags']]:
                    updated_tags = [tag['id'] for tag in marker['tags']] + [marker_source_timestamp_trade_tag['id']]
                    stash.update_scene_marker({
                        "id": marker['id'],
                        "tag_ids": updated_tags,
                    })
                    print(f"Updated marker: {data['tag_name']} for scene ID: {data['scene_id']} with new tag")
                break
        if not marker_exists:
            stash.create_scene_marker({
                "scene_id": data['scene_id'],
                "title": data['tag_name'],
                "primary_tag_id": data['tag_id'],
                "seconds": data['start_time'],
                "tag_ids": [marker_source_timestamp_trade_tag['id']],
            })
            print(f"Created marker: {data['tag_name']} for scene ID: {data['scene_id']}")
    
    refreshed_scene = stash.find_scene(scene_id)
    current_tag_ids = [tag['id'] for tag in refreshed_scene['tags']]
    if markers_timestamp_trade_saved_tag['id'] not in current_tag_ids:
        current_tag_ids.append(markers_timestamp_trade_saved_tag['id'])
        stash.update_scene({
            "id": scene_id,
            "tag_ids": current_tag_ids
        })
        print(f"Added 'Markers: Timestamp.trade: Saved' tag to scene ID: {scene_id}, Title: {existing_scene['title']}")
    
    processed_scenes += 1
